In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
data = pd.read_csv('my_agam_aging_expr_timeseries_Jun23_2023.tsv',sep='\t')
data.head()

,transcription,strainID,age,geneID,transcriptID,experimentID
0,10.617924,0,14,AGAP005098,AGAP005098-RA,1
1,9.614043,0,14,AGAP005098,AGAP005098-RA,2
2,9.930478,0,14,AGAP005098,AGAP005098-RA,3
3,10.160402,1,14,AGAP005098,AGAP005098-RA,4
4,10.482997,1,14,AGAP005098,AGAP005098-RA,5


In [3]:
# First, we need to compute the variance of gene expression for each gene at each age.
# This involves grouping the data by 'geneID' and 'age', then calculating the variance of the 'transcription' values.

# Grouping the data by 'geneID' and 'age' and calculating the variance
gene_expression_variance = data.groupby(['geneID', 'age', 'strainID'])['transcription'].var()

# Resetting the index to make 'geneID', 'age', and 'strainID' columns again
gene_expression_variance = gene_expression_variance.reset_index()

gene_expression_variance.rename(columns={'transcription': 'variance'}, inplace=True)

# Displaying the first few rows of the newly created dataframe
gene_expression_variance.head()


,geneID,age,strainID,variance
0,AGAP000002,14,0,0.000509
1,AGAP000002,14,1,0.055051
2,AGAP000002,14,2,0.000855
3,AGAP000002,14,3,0.000642
4,AGAP000002,21,0,0.037857


In [8]:
model_with_strain = smf.mixedlm("variance ~ age", data=gene_expression_variance, 
                                groups=gene_expression_variance["geneID"], 
                                re_formula="~1", 
                                vc_formula={"strainID": "0 + C(strainID)"})
result_with_strain = model_with_strain.fit()

# Print the summary of the model with strain effect
print(result_with_strain.summary())

C:\Users\86187\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: variance   
No. Observations: 213282  Method:             REML       
No. Groups:       12546   Scale:              0.1185     
Min. group size:  17      Log-Likelihood:     -98803.5904
Max. group size:  17      Converged:          Yes        
Mean group size:  17.0                                   
---------------------------------------------------------
                Coef. Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------
Intercept       0.192    0.004 42.909 0.000  0.183  0.200
age             0.001    0.000 10.133 0.000  0.001  0.001
Group Var       0.189    0.007                           
strainID Var    0.004    0.001                           



In [9]:
result_with_strain.summary().tables[1]
# Group Var (Random Effect for GeneID)
# Coef (0.189): This is the estimated variance of the random intercepts for genes, 
# indicating how much gene expression variance varies from one gene to another.

# StrainID Var (Random Effect for StrainID)
# Similarly as Group Var

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
Intercept,0.192,0.004,42.909,0.000,0.183,0.200
age,0.001,0.000,10.133,0.000,0.001,0.001
Group Var,0.189,0.007,,,,
strainID Var,0.004,0.001,,,,


In [18]:
# Extracting random effects
random_effects = result_with_strain.random_effects
# Converting the dictionary of random effects to a DataFrame
random_effects_df = pd.DataFrame.from_dict(random_effects, orient='index')
random_effects_df.head()

#The column "Group" in random_effects_df represents these slopes over age for each group(gene in our context).

,Group,strainID[C(strainID)[0]],strainID[C(strainID)[1]],strainID[C(strainID)[2]],strainID[C(strainID)[3]]
AGAP000002,-0.184895,-0.002178,-0.000039,-0.000695,-0.001402
AGAP000005,-0.117533,-0.008847,-0.005187,0.016571,-0.005279
AGAP000007,0.013649,0.005128,-0.009891,0.012181,-0.007100
AGAP000008,-0.100636,-0.009514,-0.009032,0.001944,0.014254
AGAP000009,-0.100822,-0.015004,-0.007024,0.010959,0.008715


I also conducted a Gene Ontology analysis using a tool named DAVID.Below are the results for genes with significantly increasing variance over age.

Here's a brief overview of the key columns in the DAVID summary chart:

1. **Category**: This indicates the type of Gene Ontology (GO) term or other categorization, such as GOTERM_BP_DIRECT (Biological Process), GOTERM_MF_DIRECT (Molecular Function), or GOTERM_CC_DIRECT (Cellular Component).

2. **Term**: The specific GO term or pathway identified in the analysis. It usually contains the GO ID and a brief description (e.g., "GO:0005549~odorant binding").

3. **Count**: The number of genes from the list that are associated with this particular term.

4. **%**: The percentage of genes from the list that fall into this category.

5. **PValue**: The p-value indicating the statistical significance of the association between the gene list and the GO term. Lower values indicate a more significant association.

6. **Genes**: The specific genes from the list that are associated with this GO term.

7. **List Total**: The total number of genes in the list.

8. **Pop Hits**: The number of genes in the background population (like the entire genome) that are associated with this term.

9. **Pop Total**: The total number of genes in the background population.

10. **Fold Enrichment**: Indicates how much more common the GO term is in your list compared to the background population. Higher values suggest a stronger association.

11. **Bonferroni, Benjamini, FDR**: These are different methods of adjusting the p-value for multiple testing. They help control the false discovery rate when many hypotheses are tested simultaneously.

- Look for trends or common themes among the highly significant GO terms. Do they cluster around certain biological processes or functions?
- Identify the most enriched pathways or processes. These are the ones where the genes are overrepresented compared to what would be expected by chance.
- Since we're studying aging, we are particularly interested in GO terms related to cell senescence, DNA repair, or metabolic processes.


In [19]:
file_path = 'chart_Go.txt'
go_data = pd.read_csv(file_path, sep="\t")

go_data.head() 


,Category,Term,Count,%,PValue,Genes,List Total,Pop Hits,Pop Total,Fold Enrichment,Bonferroni,Benjamini,FDR
0,GOTERM_BP_DIRECT,GO:0050911~detection of chemical stimulus invo...,67,1.686383,8.183479e-31,"AGAP009520, AGAP012854, AGAP009640, AGAP004355...",1554,80,7042,3.795158,8.478084e-28,8.469901e-28,8.420800e-28
1,GOTERM_MF_DIRECT,GO:0005549~odorant binding,101,2.542160,4.716560e-29,"AGAP009520, AGAP009640, AGAP012659, AGAP012658...",1885,149,7787,2.800231,3.358191e-26,3.358191e-26,3.292159e-26
2,GOTERM_MF_DIRECT,GO:0004984~olfactory receptor activity,68,1.711553,3.673612e-28,"AGAP009520, AGAP012854, AGAP009640, AGAP004355...",1885,82,7787,3.425736,2.615612e-25,1.307806e-25,1.282091e-25
3,GOTERM_BP_DIRECT,GO:0050909~sensory perception of taste,46,1.157815,2.196892e-22,"AGAP010195, AGAP009803, AGAP006875, AGAP001120...",1554,53,7042,3.933027,2.275980e-19,1.136892e-19,1.130301e-19
4,GOTERM_CC_DIRECT,GO:0005886~plasma membrane,276,6.946892,1.321277e-19,"AGAP012891, AGAP012656, AGAP004355, AGAP009803...",1945,731,8256,1.602661,5.245468e-17,5.245468e-17,5.205830e-17


In [21]:
# Analysis of the DAVID summary data

# Filter the data to focus on highly significant terms
# We'll use a p-value threshold of 0.05 and fold enrichment greater than 1.5 as an example
significant_terms = go_data[(go_data['PValue'] < 0.05) & (go_data['Fold Enrichment'] > 1.5)]

# Analysis 1: Identifying the most enriched GO terms
most_enriched_terms = significant_terms.sort_values(by='Fold Enrichment', ascending=False).head(10)

most_enriched_terms

,Category,Term,Count,%,PValue,Genes,List Total,Pop Hits,Pop Total,Fold Enrichment,Bonferroni,Benjamini,FDR
41,GOTERM_BP_DIRECT,GO:0046177~D-gluconate catabolic process,6,0.151019,2.543313e-03,"AGAP010329, AGAP004687, AGAP012497, AGAP004197...",1554,6,7042,4.531532,9.285122e-01,2.632328e-01,2.617069e-01
14,GOTERM_MF_DIRECT,GO:0004616~phosphogluconate dehydrogenase (dec...,12,0.302039,1.520606e-06,"AGAP010329, AGAP004687, AGAP004676, AGAP012497...",1885,12,7787,4.131034,1.082086e-03,2.706678e-04,2.653457e-04
3,GOTERM_BP_DIRECT,GO:0050909~sensory perception of taste,46,1.157815,2.196892e-22,"AGAP010195, AGAP009803, AGAP006875, AGAP001120...",1554,53,7042,3.933027,2.275980e-19,1.136892e-19,1.130301e-19
53,GOTERM_BP_DIRECT,"GO:0009051~pentose-phosphate shunt, oxidative ...",6,0.151019,7.280058e-03,"AGAP010329, AGAP004687, AGAP012497, AGAP004197...",1554,7,7042,3.884170,9.994842e-01,4.709287e-01,4.681987e-01
0,GOTERM_BP_DIRECT,GO:0050911~detection of chemical stimulus invo...,67,1.686383,8.183479e-31,"AGAP009520, AGAP012854, AGAP009640, AGAP004355...",1554,80,7042,3.795158,8.478084e-28,8.469901e-28,8.420800e-28
37,GOTERM_BP_DIRECT,GO:0006098~pentose-phosphate shunt,8,0.201359,1.591628e-03,"AGAP010329, AGAP004676, AGAP012542, AGAP012583...",1554,10,7042,3.625225,8.079962e-01,1.830372e-01,1.819761e-01
49,GOTERM_BP_DIRECT,GO:0050916~sensory perception of sweet taste,7,0.176189,5.143217e-03,"AGAP003255, AGAP003256, AGAP003258, AGAP003253...",1554,9,7042,3.524525,9.952143e-01,3.919050e-01,3.896331e-01
2,GOTERM_MF_DIRECT,GO:0004984~olfactory receptor activity,68,1.711553,3.673612e-28,"AGAP009520, AGAP012854, AGAP009640, AGAP004355...",1885,82,7787,3.425736,2.615612e-25,1.307806e-25,1.282091e-25
70,GOTERM_BP_DIRECT,GO:0060285~cilium-dependent cell motility,6,0.151019,1.590046e-02,"AGAP002143, AGAP001293, AGAP006596, AGAP003805...",1554,8,7042,3.398649,9.999999e-01,7.836657e-01,7.791227e-01
42,GOTERM_MF_DIRECT,GO:0004620~phospholipase activity,8,0.201359,2.844505e-03,"AGAP009956, AGAP005971, AGAP006396, AGAP011512...",1885,10,7787,3.304828,8.684244e-01,1.125160e-01,1.103036e-01


These terms suggest that processes related to sensory perception (taste and smell), metabolism (particularly involving pentose-phosphate shunt and glu conate catabolism), and certain molecular functions like olfactory receptor activity and phospholipase activity are highly represented among genes with increased variance in expression with age.

In [22]:
# Analysis 2
# Extracting and analyzing the genes from the DAVID summary data to identify frequently occurring genes

# Extracting the genes from each GO term
genes_list = go_data['Genes'].str.split(',').tolist()
# Flattening the list
all_genes = [gene.strip() for sublist in genes_list for gene in sublist]

# Counting the frequency of each gene
gene_frequency = pd.Series(all_genes).value_counts()

# Display the most frequently occurring genes
most_frequent_genes = gene_frequency.head(10)  # Top 10 for example
most_frequent_genes


AGAP010790    11
AGAP005637     9
AGAP005636     9
AGAP005638     9
AGAP009999     9
AGAP006221     9
AGAP006224     9
AGAP011859     9
AGAP009137     8
AGAP009519     8
dtype: int64

These genes have frequent association with the enriched GO terms in the DAVID summary. They might play significant roles in the biological processes or pathways impacted by aging, as indicated by their repeated occurrence across different GO categories.